# 감성 분석에 머신 러닝 적용

In [3]:
# 영화 리뷰 데이터셋
# http://ai.stanford.edu/~amaas/data/sentiment/

import pyprind  # 진행 과정 예상 시간 추측 라이브러리
import pandas as pd
import os

basepath = './aclImdb_v1/aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:02


In [4]:
# csv 파일로 저장

import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))  # 데이터프레임 섞기
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [1]:
# 저장 확인
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [2]:
# 데이터 프레임 크기 출력

df.shape

(50000, 2)

# BoW 모델 소개

In [3]:
# 단어를 특성 벡터로 변환

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()  
# 기본적으로 1-gram(unigram). 
# ngram_range 매개변수 사용하면 변경 가능. ex) ngram_range = (2, 2) => 2-gram

docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [4]:
# 어휘 사전 내용 출력

print(count.vocabulary_)  
print(type(count.vocabulary_))  # 딕셔너리 타입

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
<class 'dict'>


In [5]:
# 특성 벡터 출력

print(bag.toarray())  

# 특성 벡터의 각 인덱스는 어휘 사전 딕셔너리에 저장된 정수 값에 해당함
# ['and'의 수, 'is'의 수, 'one'의 수, ...]

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [6]:
# tf-idf를 사용하여 단어 적합성 평가

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',  # 'l2' 정규화 적용
                         smooth_idf=True)  # False => 분모와 분자에 1 더하지 않음
np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


## 텍스트 데이터 정제

In [7]:
# 영화 리뷰 데이터셋의 첫 번째 문서 마지막 50개 문자 출력

df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [8]:
# 이모티콘 문자를 제외한 모든 구두점 기호 삭제

import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)  # 모든 HTML 마크업 삭제
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)  # 이모티콘을 찾아 저장
    text = (re.sub('[\W]+', ' ', text.lower()) +  # 텍스트 소문자로 변경. 단어가 아닌 문자 제거
            ' '.join(emoticons).replace('-', ''))  # 이모티콘을 처리 완료된 문자열 끝에 추가. '-' 삭제
    return text

In [9]:
# 함수 동작 확인

preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [10]:
preprocessor('</a>This :) is :( a test :-)!')

'this is a test :) :( :)'

In [11]:
# 데이터프레임에 있는 모든 영화 리뷰에 preprocessor 함수 적용

df['review'] = df['review'].apply(preprocessor)

## 문서를 토큰으로 나누기

In [12]:
# 1. 공백 문자를 기준으로 개별 단어로 나누기

def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [13]:
# 2. 어간 추출(stemming) - 포터 어간 추출기(Porter stemmer)

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

# 이 외에도 스노우볼 어간 추출기(Snowball stemmer), 랭커스터 어간 추출기(Lancaster stemmer) 존재

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [14]:
# 불용어 제거

import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 문서 분류를 위한 로지스틱 회귀 모델 훈련

In [16]:
# 훈련 세트 25000개, 테스트 세트 25000개 분리

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
# GridSearchCV 5-겹 계층별 교차 검증 사용
# 최적의 매개변수 조합 찾기

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer + TfidfTransformer = TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
             ]
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(solver='liblinear', random_state=0))
                    ])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [29]:
# 최적의 매개변수 조합 출력
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)

# 정확도 출력
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_

print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

AttributeError: 'GridSearchCV' object has no attribute 'best_param_'

# 대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습

In [30]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

# 텍스트 데이터 정제. 불용어 제외한 단어 토큰으로 분리
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower() +
                  ' '.join(emoticons).replace('-', ''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [31]:
# 한 번에 문서 하나씩 읽어서 반환하는 제너레이터(generator) 함수

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [32]:
# stream_docs 함수 테스트

next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [33]:
# stream_docs 함수에서 문서를 읽어 size 매개변수에서 지정한 만큼 문서 반환하는 함수

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [34]:
# 외부 메모리 학습에 CountVectorizer, TfidfVectorizer 사용 못함
# HashingVectorizer 사용

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [35]:
# 외부 메모리 학습 시작

import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:36


In [36]:
# 모델 성능 평가

X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)

print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [37]:
# test 데이터 활용 문서 사용해서 모델 업데이트

clf = clf.partial_fit(X_test, y_test)

# 잠재 디리클레 할당(LDA)을 사용한 토픽 모델링

In [38]:
# 데이터셋 읽기

import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [39]:
# LDA 입력으로 넣을 BoW 행렬 생성

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1, max_features=5000)
# max_df=.1 => 단어 최대 문서 빈도 10%로 제한. 자주 등장하는 단어 제외함.
# max_features => 가장 자주 등장하는 단어 수 제한 => 데이터셋 차원 제한해서 LDA 성능 향상
# 하이퍼파라미터 튜닝으로 매개변수 값 설정하기

X = count.fit_transform(df['review'].values)

In [40]:
# 10개의 토픽 추정하도록 학습

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')  
# 'batch' => 가능한 모든 훈련데이터 사용해서 학습
# 'online' => 온라인 학습이나 미니 배치 학습과 유사

X_topics = lda.fit_transform(X)  # (50000, 10)

KeyboardInterrupt: 

In [ ]:
# components_: 10개 토픽에 대해 오름차순으로 단어 중요도 담은 행렬 저장

lda.components_.shape

In [ ]:
# 10개의 토픽에서 가장 중요한 단어 5개씩 출력

n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print('토픽 %d:' % (topic_idx + 1))
    print(' '.join([feature_names[i] for i in topic.argsort()\
                    [:-n_top_words - 1:-1]]))
    
# 각 단어를 기반으로 토픽 추측

In [ ]:
# 카테고리가 잘 나누어 졌는지 확인

horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화#%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')